In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
df_books = pd.read_csv("data/Books.csv", header = 0)
df_ratings = pd.read_csv("data/Ratings.csv", header = 0)
df_users = pd.read_csv("data/Users.csv", header = 0)

/var/folders/g6/m9mjx8gn46xg9qvcz4921f_c0000gn/T/ipykernel_4252/2563372557.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv("data/Books.csv", header = 0)


### EDA: df_books

1. **ISBN**: unique identifier assigned to each edition and variation of a book
2. **Book-Title**: book title
3. **Book-Author**: author
4. **Year-Of-Publication**: 4 digit year
5. **Publisher**: Publisher
6. **Image-URL-S**: URL to book image (small)
7. **Image-URL-M**: URL to book image (medium)
8. **Image-URL-L**: URL to book image (large)

In [3]:
df_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
print("Columns:")
print(df_books.columns)

print("--------------------------------------------------")

print("Shape:", df_books.shape)

print("--------------------------------------------------")

print("NULL values:", df_books.isnull().sum())

print("--------------------------------------------------")
print("Datatypes:", df_books.dtypes)

Columns:
Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')
--------------------------------------------------
Shape: (271360, 8)
--------------------------------------------------
NULL values: ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64
--------------------------------------------------
Datatypes: ISBN                   object
Book-Title             object
Book-Author            object
Year-Of-Publication    object
Publisher              object
Image-URL-S            object
Image-URL-M            object
Image-URL-L            object
dtype: object


### EDA: df_ratings

More rows in this dataset, as there can be multiple ratings for the same book

In [5]:
df_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
print("Columns:")
print(df_ratings.columns)

print("--------------------------------------------------")

print("Shape:", df_ratings.shape)

print("--------------------------------------------------")

print("NULL values:", df_ratings.isnull().sum())

print("--------------------------------------------------")
print("Datatypes:", df_ratings.dtypes)

Columns:
Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')
--------------------------------------------------
Shape: (1149780, 3)
--------------------------------------------------
NULL values: User-ID        0
ISBN           0
Book-Rating    0
dtype: int64
--------------------------------------------------
Datatypes: User-ID         int64
ISBN           object
Book-Rating     int64
dtype: object


In [7]:
# df_ratings rows > df_books rows -----> check if it's because of multiple ratings on the same book
df_ratings[df_ratings.duplicated(subset = ["ISBN"])]

,User-ID,ISBN,Book-Rating
91,276798,3442437407,0
295,276861,360893541X,9
296,276861,3608935428,9
297,276861,3608935436,9
397,276904,0440414121,10
...,...,...,...
1149774,276704,0876044011,0
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10


In [8]:
df_ratings.loc[df_ratings["ISBN"] == "360893541X"] # ex. pull 360893541X as an example -> theory confirmed

,User-ID,ISBN,Book-Rating
216,276847,360893541X,10
295,276861,360893541X,9
335199,79687,360893541X,10
671061,163286,360893541X,10
675078,164307,360893541X,8
874274,211460,360893541X,10
911935,222150,360893541X,3
982323,236186,360893541X,0
1139923,274111,360893541X,0


In [9]:
df_ratings.loc[df_ratings["ISBN"] == "0195153448"]

,User-ID,ISBN,Book-Rating
9561,2,0195153448,0


### EDA: df_users

In [10]:
df_users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [11]:
print("Columns:")
print(df_users.columns)

print("--------------------------------------------------")

print("Shape:", df_users.shape)

print("--------------------------------------------------")

print("NULL values:", df_users.isnull().sum())

print("--------------------------------------------------")
print("Datatypes:", df_users.dtypes)

Columns:
Index(['User-ID', 'Location', 'Age'], dtype='object')
--------------------------------------------------
Shape: (278858, 3)
--------------------------------------------------
NULL values: User-ID          0
Location         0
Age         110762
dtype: int64
--------------------------------------------------
Datatypes: User-ID       int64
Location     object
Age         float64
dtype: object


## Data Cleaning

In [12]:
########### df_books cleaning
## drop image columns
df_books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis = 1, inplace = True)

In [13]:
## drop NULL values in df_books
df_books.dropna(inplace = True)
print(df_books.isnull().sum())
print("----------------------")
print(df_books.shape)

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
dtype: int64
----------------------
(271357, 5)


In [14]:
df_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [15]:
# checking for duplicates:
print("books duplicates:", df_books.duplicated().sum())
print("ratings duplicates:", df_ratings.duplicated().sum())
print("users duplicates:", df_users.duplicated().sum())

books duplicates: 0
ratings duplicates: 0
users duplicates: 0


## Potential Factors in Book Recommendation

This is from personal experience:
- Similar book genres --> KNN model will help with that
- Books with HIGH Ratings
- Books with MORE ratings
- People who've read more books (so they can recommend better books)

## Filtering

### Filter out books with low number of ratings - Want books with MORE ratings

In [16]:
df_low_num_rating_books = df_ratings["ISBN"].value_counts().reset_index()
df_low_num_rating_books.columns = ["ISBN", "Rating Count"]
df_low_num_rating_books.sort_values("Rating Count", ascending = False)
df_low_num_rating_books.head()

,ISBN,Rating Count
0,0971880107,2502
1,0316666343,1295
2,0385504209,883
3,0060928336,732
4,0312195516,723


In [17]:
# keep books with more ratings
# let low number of ratings for a book be 100 ratings
high_rating_count = df_low_num_rating_books[df_low_num_rating_books["Rating Count"] < 100]
df_ratings_new = df_ratings[~df_ratings["ISBN"].isin(high_rating_count["ISBN"])]

In [18]:
df_ratings_new.head()

,User-ID,ISBN,Book-Rating
2,276727,0446520802,0
8,276744,038550120X,7
10,276746,0425115801,0
11,276746,0449006522,0
12,276746,0553561618,0


In [19]:
df_ratings_new.shape

(138461, 3)

### Filter out users with low activity - Want recs from people who've read more books (so they can recommend better books)

In [20]:
# We want users with higher activity to see their recs for better recommendation system
df_user_activity = df_ratings["User-ID"].value_counts().reset_index()
df_user_activity.columns = ["User-ID", "Activity Count"]
df_user_activity = df_user_activity.sort_values("Activity Count", ascending = False) # see most active users first
df_user_activity.head()

,User-ID,Activity Count
0,11676,13602
1,198711,7550
2,153662,6109
3,98391,5891
4,35859,5850


In [21]:
# keep users who have high activity
# let low activity for users be users who have rated less than 100 books
active_users = df_user_activity[df_user_activity["Activity Count"] < 100]

# Filter out ratings from users with less than 100 ratings
df_ratings_new = df_ratings_new[~df_ratings_new["User-ID"].isin(active_users["User-ID"])]

In [22]:
df_ratings_new.head()

,User-ID,ISBN,Book-Rating
413,276925,002542730X,10
426,276925,0316666343,0
429,276925,0385504209,8
453,276925,0804106304,0
457,276925,0971880107,0


In [23]:
df_ratings_new.shape

(65896, 3)

## Book Recommendation - KNN Model

In [24]:
# create pivot table
df_pivot = df_ratings_new.pivot_table(index = "User-ID", columns = "ISBN", values = "Book-Rating", fill_value = 0).T

In [25]:
df_pivot.head()

User-ID,254,507,882,1424,1435,1733,1903,2033,2110,2276,...,276018,276463,276680,276925,277427,277478,277639,278137,278188,278418
ISBN,,,,,,,,,,,,,,,,,,,,,
002542730X,0,0,0,0,0,0,0,0,0,0,...,0,0,0,10,10,0,0,0,0,0
0060008032,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0060096195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
006016848X,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0060173289,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
df_pivot.index = df_pivot.join(df_books.set_index('ISBN'))['Book-Title']
df_pivot = df_pivot.sort_index()

In [27]:
df_pivot.head()

User-ID,254,507,882,1424,1435,1733,1903,2033,2110,2276,...,276018,276463,276680,276925,277427,277478,277639,278137,278188,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1st to Die: A Novel,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1st to Die: A Novel,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2nd Chance,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2nd Chance,0,0,0,0,0,0,0,0,0,10,...,0,0,0,0,0,0,0,0,0,0


In [32]:
print(df_pivot.values)

[[ 9  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 8  0 10 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [72]:
X = df_pivot.values
y = df_pivot

model_knn = NearestNeighbors(n_neighbors = 5, metric = "cosine")
model_knn.fit(df_pivot.values)

NearestNeighbors(metric='cosine')

In [73]:
def get_book_recs(title):
    book = df_pivot.loc[title]
    distance, book_loc = model_knn.kneighbors([book.values], n_neighbors = 5)
    
    book_title = df_pivot.iloc[book_loc[0]].index.values
    distance_value = distance[0]
    
    df_recommended_books = pd.DataFrame({
        "Book Title" : book_title,
        "Distance Value" : distance_value
    }).sort_values(by = "Distance Value", ascending = False)
    
    print("BOOK:", title)
    print("We Recommend the Following:\n")
    print(df_recommended_books.head())

In [74]:
book1 = get_book_recs("Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))")

BOOK: Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
We Recommend the Following:

                                          Book Title  Distance Value
4       Harry Potter and the Goblet of Fire (Book 4)        0.741137
3  Harry Potter and the Order of the Phoenix (Boo...        0.740948
2  Harry Potter and the Prisoner of Azkaban (Book 3)        0.701142
1   Harry Potter and the Chamber of Secrets (Book 2)        0.598446
0  Harry Potter and the Sorcerer's Stone (Harry P...        0.000000


In [75]:
book2 = get_book_recs("The Queen of the Damned (Vampire Chronicles (Paperback))")

BOOK: The Queen of the Damned (Vampire Chronicles (Paperback))
We Recommend the Following:

                                          Book Title  Distance Value
4   The Witching Hour (Lives of the Mayfair Witches)        0.803166
3                         Interview with the Vampire        0.716940
2   The Vampire Lestat (Vampire Chronicles, Book II)        0.529230
1  The Tale of the Body Thief (Vampire Chronicles...        0.477580
0  The Queen of the Damned (Vampire Chronicles (P...        0.000000


## Book Recommendation - Collaborative Filtering